In [5]:
import src.dataPipeline as dataPipeline
import importlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
# Evaluating the model
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error,r2_score ,make_scorer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

importlib.reload(dataPipeline)


<module 'src.dataPipeline' from 'C:\\Users\\wartm\\Documents\\FHNW\\immo_challenge\\src\\dataPipeline.py'>

In [6]:
def calculate_metrics(X_train, y_test, y_pred):
    n = len(y_test)  # Number of observations
    k = X_train.shape[1]  # Number of predictors
    r2 = round(r2_score(y_test, y_pred), 4)
    R2_adjusted = round(1 - (1 - r2) * (n - 1) / (n - k - 1), 4)
    mape = round(mean_absolute_percentage_error(y_test, y_pred) * 100, 4)
    return r2, R2_adjusted, mape

In [7]:
dp = dataPipeline.DataPipeline()
df = dp.runPipeline(
    filePath="../data/immo_data_202208_v2.csv",
    imputer=None,
    normalizeAndStandardize= False,
    get_dummies = False
)

C:\Users\wartm\Documents\FHNW\immo_challenge\src\dataPipeline.py:41: DtypeWarning: Columns (3,4,5,6,11,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,47,49,50,107,110,114,115,116,119,120,121,124,125,126,128,131,132) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data = pd.read_csv(filePath)
C:\Users\wartm\Documents\FHNW\immo_challenge\src\dataPipeline.py:64: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ]].bfill(axis=1)['Space extracted']
C:\Users\wartm\Documents\FHNW\immo_challenge\src\dataPipeline.py:69: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in 

In [8]:
#show all columns
pd.set_option('display.max_columns', None)

In [9]:
df.head()

,Availability,Floor,detail_responsive#surface_usable,Floor_space_merged,ForestDensityL,ForestDensityM,ForestDensityS,NoisePollutionRailwayL,NoisePollutionRailwayM,NoisePollutionRailwayS,NoisePollutionRoadL,NoisePollutionRoadM,NoisePollutionRoadS,PopulationDensityL,PopulationDensityM,PopulationDensityS,RiversAndLakesL,RiversAndLakesM,RiversAndLakesS,WorkplaceDensityL,WorkplaceDensityM,WorkplaceDensityS,distanceToTrainStation,gde_area_agriculture_percentage,gde_area_forest_percentage,gde_area_nonproductive_percentage,gde_area_settlement_percentage,gde_average_house_hold,gde_empty_apartments,gde_foreigners_percentage,gde_new_homes_per_1000,gde_politics_bdp,gde_politics_cvp,gde_politics_evp,gde_politics_fdp,gde_politics_glp,gde_politics_gps,gde_politics_pda,gde_politics_rights,gde_politics_sp,gde_politics_svp,gde_pop_per_km2,gde_population,gde_private_apartments,gde_social_help_quota,gde_tax,gde_workers_sector1,gde_workers_sector2,gde_workers_sector3,gde_workers_total,price_cleaned,Space extracted,type_unified,Plot_area_unified,No. of rooms:,Last refurbishment:,Year built:,Number of floors:,region_group
0,On request,4.0,NaN,NaN,0.511176,0.286451,0.090908,0.0,0.0,0.0,0.058298,0.067048,0.103850,0.092914,0.209530,0.366674,0.082170,0.001811,0.011871,0.030169,0.052120,0.098951,3.038467,30.676329,51.449275,4.589372,13.285024,2.23,1.994681,9.255663,4.739336,5.873715,4.579662,3.359031,18.355360,6.057269,7.066814,NaN,0.220264,20.392805,30.809471,376.829268,1545.0,686.0,2.234259,5.89,14.0,9.0,308.0,331.0,1150000.0,100.0,penthouse,NaN,5.0,NaN,NaN,NaN,2.0
1,On request,NaN,NaN,242.0,0.511176,0.286451,0.090908,0.0,0.0,0.0,0.058298,0.067048,0.103850,0.092914,0.209530,0.366674,0.082170,0.001811,0.011871,0.030169,0.052120,0.098951,3.038467,30.676329,51.449275,4.589372,13.285024,2.23,1.994681,9.255663,4.739336,5.873715,4.579662,3.359031,18.355360,6.057269,7.066814,NaN,0.220264,20.392805,30.809471,376.829268,1545.0,686.0,2.234259,5.89,14.0,9.0,308.0,331.0,1420000.0,156.0,terrace-house,222.0,5.0,NaN,NaN,NaN,2.0
2,Immediately,2.0,NaN,NaN,0.163362,0.095877,0.001911,0.0,0.0,0.0,0.334957,0.381257,0.297575,0.325887,0.393783,0.635194,0.154274,0.188229,0.000000,0.172646,0.163850,0.165830,0.909587,11.354420,32.197891,7.137064,49.310624,2.01,2.023799,21.358623,3.814582,3.633134,5.324421,3.782202,18.089552,7.899807,8.851305,NaN,0.735032,26.515854,22.662290,1704.700162,21036.0,10149.0,3.549010,6.05,37.0,3092.0,30364.0,33493.0,720000.0,93.0,penthouse,NaN,5.0,NaN,NaN,NaN,2.0
3,On request,NaN,NaN,257.0,0.511176,0.286451,0.090908,0.0,0.0,0.0,0.058298,0.067048,0.103850,0.092914,0.209530,0.366674,0.082170,0.001811,0.011871,0.030169,0.052120,0.098951,3.038467,30.676329,51.449275,4.589372,13.285024,2.23,1.994681,9.255663,4.739336,5.873715,4.579662,3.359031,18.355360,6.057269,7.066814,NaN,0.220264,20.392805,30.809471,376.829268,1545.0,686.0,2.234259,5.89,14.0,9.0,308.0,331.0,1430000.0,154.0,detached-house,370.0,5.0,NaN,NaN,NaN,2.0
4,On request,0.0,NaN,NaN,0.333865,0.279276,0.145835,0.0,0.0,0.0,0.133498,0.132933,0.235917,0.190986,0.136984,0.204549,0.109586,0.141473,0.091805,0.046950,0.038008,0.055509,1.460245,33.137090,49.705635,1.177460,15.979815,2.28,0.691563,15.901990,1.160862,5.217740,5.728026,5.006679,19.158429,6.502805,7.477959,NaN,0.892332,20.459524,27.590168,511.008403,6081.0,2638.0,1.708126,6.30,65.0,349.0,941.0,1355.0,995000.0,142.0,flat,NaN,5.0,NaN,NaN,NaN,2.0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20786 entries, 0 to 22480
Data columns (total 59 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Availability                       12028 non-null  object 
 1   Floor                              9640 non-null   float64
 2   detail_responsive#surface_usable   104 non-null    float64
 3   Floor_space_merged                 2591 non-null   float64
 4   ForestDensityL                     20786 non-null  float64
 5   ForestDensityM                     20786 non-null  float64
 6   ForestDensityS                     20786 non-null  float64
 7   NoisePollutionRailwayL             20786 non-null  float64
 8   NoisePollutionRailwayM             20786 non-null  float64
 9   NoisePollutionRailwayS             20786 non-null  float64
 10  NoisePollutionRoadL                20786 non-null  float64
 11  NoisePollutionRoadM                20786 non-null  float64


In [11]:
#Categorical columns
cat_col =["Availability","type_unified","region_group"]

In [12]:
df[cat_col] = df[cat_col].astype('category')

In [13]:
df[cat_col].isna().sum()

Availability    8758
type_unified       0
region_group       0
dtype: int64

In [14]:
df.head()

,Availability,Floor,detail_responsive#surface_usable,Floor_space_merged,ForestDensityL,ForestDensityM,ForestDensityS,NoisePollutionRailwayL,NoisePollutionRailwayM,NoisePollutionRailwayS,NoisePollutionRoadL,NoisePollutionRoadM,NoisePollutionRoadS,PopulationDensityL,PopulationDensityM,PopulationDensityS,RiversAndLakesL,RiversAndLakesM,RiversAndLakesS,WorkplaceDensityL,WorkplaceDensityM,WorkplaceDensityS,distanceToTrainStation,gde_area_agriculture_percentage,gde_area_forest_percentage,gde_area_nonproductive_percentage,gde_area_settlement_percentage,gde_average_house_hold,gde_empty_apartments,gde_foreigners_percentage,gde_new_homes_per_1000,gde_politics_bdp,gde_politics_cvp,gde_politics_evp,gde_politics_fdp,gde_politics_glp,gde_politics_gps,gde_politics_pda,gde_politics_rights,gde_politics_sp,gde_politics_svp,gde_pop_per_km2,gde_population,gde_private_apartments,gde_social_help_quota,gde_tax,gde_workers_sector1,gde_workers_sector2,gde_workers_sector3,gde_workers_total,price_cleaned,Space extracted,type_unified,Plot_area_unified,No. of rooms:,Last refurbishment:,Year built:,Number of floors:,region_group
0,On request,4.0,NaN,NaN,0.511176,0.286451,0.090908,0.0,0.0,0.0,0.058298,0.067048,0.103850,0.092914,0.209530,0.366674,0.082170,0.001811,0.011871,0.030169,0.052120,0.098951,3.038467,30.676329,51.449275,4.589372,13.285024,2.23,1.994681,9.255663,4.739336,5.873715,4.579662,3.359031,18.355360,6.057269,7.066814,NaN,0.220264,20.392805,30.809471,376.829268,1545.0,686.0,2.234259,5.89,14.0,9.0,308.0,331.0,1150000.0,100.0,penthouse,NaN,5.0,NaN,NaN,NaN,2.0
1,On request,NaN,NaN,242.0,0.511176,0.286451,0.090908,0.0,0.0,0.0,0.058298,0.067048,0.103850,0.092914,0.209530,0.366674,0.082170,0.001811,0.011871,0.030169,0.052120,0.098951,3.038467,30.676329,51.449275,4.589372,13.285024,2.23,1.994681,9.255663,4.739336,5.873715,4.579662,3.359031,18.355360,6.057269,7.066814,NaN,0.220264,20.392805,30.809471,376.829268,1545.0,686.0,2.234259,5.89,14.0,9.0,308.0,331.0,1420000.0,156.0,terrace-house,222.0,5.0,NaN,NaN,NaN,2.0
2,Immediately,2.0,NaN,NaN,0.163362,0.095877,0.001911,0.0,0.0,0.0,0.334957,0.381257,0.297575,0.325887,0.393783,0.635194,0.154274,0.188229,0.000000,0.172646,0.163850,0.165830,0.909587,11.354420,32.197891,7.137064,49.310624,2.01,2.023799,21.358623,3.814582,3.633134,5.324421,3.782202,18.089552,7.899807,8.851305,NaN,0.735032,26.515854,22.662290,1704.700162,21036.0,10149.0,3.549010,6.05,37.0,3092.0,30364.0,33493.0,720000.0,93.0,penthouse,NaN,5.0,NaN,NaN,NaN,2.0
3,On request,NaN,NaN,257.0,0.511176,0.286451,0.090908,0.0,0.0,0.0,0.058298,0.067048,0.103850,0.092914,0.209530,0.366674,0.082170,0.001811,0.011871,0.030169,0.052120,0.098951,3.038467,30.676329,51.449275,4.589372,13.285024,2.23,1.994681,9.255663,4.739336,5.873715,4.579662,3.359031,18.355360,6.057269,7.066814,NaN,0.220264,20.392805,30.809471,376.829268,1545.0,686.0,2.234259,5.89,14.0,9.0,308.0,331.0,1430000.0,154.0,detached-house,370.0,5.0,NaN,NaN,NaN,2.0
4,On request,0.0,NaN,NaN,0.333865,0.279276,0.145835,0.0,0.0,0.0,0.133498,0.132933,0.235917,0.190986,0.136984,0.204549,0.109586,0.141473,0.091805,0.046950,0.038008,0.055509,1.460245,33.137090,49.705635,1.177460,15.979815,2.28,0.691563,15.901990,1.160862,5.217740,5.728026,5.006679,19.158429,6.502805,7.477959,NaN,0.892332,20.459524,27.590168,511.008403,6081.0,2638.0,1.708126,6.30,65.0,349.0,941.0,1355.0,995000.0,142.0,flat,NaN,5.0,NaN,NaN,NaN,2.0


In [15]:
df.loc[df['Space extracted'] < 5, 'Space extracted'] = np.nan
df.loc[df["Floor"] >= 41, "Floor"] = np.nan

#Filling Floor for House types with zeros
house_types = [
    'detached-house', 'villa', 'semi-detached-house', 'terrace-house',
    'chalet', 'farmhouse', 'rustico', 'castle', 'detached-secondary-suite'
]
df.loc[
    (df['type_unified'].isin(house_types)) & (df['Floor'].isna()),
    'Floor'
] = 0
#Fill na with 0
df["detail_responsive#surface_usable"] = df["detail_responsive#surface_usable"].fillna(0)
df["Number of floors:"] = df["Number of floors:"].fillna(1)
df["Plot_area_unified"] = df["Plot_area_unified"].fillna(0)

In [16]:
df.head()

,Availability,Floor,detail_responsive#surface_usable,Floor_space_merged,ForestDensityL,ForestDensityM,ForestDensityS,NoisePollutionRailwayL,NoisePollutionRailwayM,NoisePollutionRailwayS,NoisePollutionRoadL,NoisePollutionRoadM,NoisePollutionRoadS,PopulationDensityL,PopulationDensityM,PopulationDensityS,RiversAndLakesL,RiversAndLakesM,RiversAndLakesS,WorkplaceDensityL,WorkplaceDensityM,WorkplaceDensityS,distanceToTrainStation,gde_area_agriculture_percentage,gde_area_forest_percentage,gde_area_nonproductive_percentage,gde_area_settlement_percentage,gde_average_house_hold,gde_empty_apartments,gde_foreigners_percentage,gde_new_homes_per_1000,gde_politics_bdp,gde_politics_cvp,gde_politics_evp,gde_politics_fdp,gde_politics_glp,gde_politics_gps,gde_politics_pda,gde_politics_rights,gde_politics_sp,gde_politics_svp,gde_pop_per_km2,gde_population,gde_private_apartments,gde_social_help_quota,gde_tax,gde_workers_sector1,gde_workers_sector2,gde_workers_sector3,gde_workers_total,price_cleaned,Space extracted,type_unified,Plot_area_unified,No. of rooms:,Last refurbishment:,Year built:,Number of floors:,region_group
0,On request,4.0,0.0,NaN,0.511176,0.286451,0.090908,0.0,0.0,0.0,0.058298,0.067048,0.103850,0.092914,0.209530,0.366674,0.082170,0.001811,0.011871,0.030169,0.052120,0.098951,3.038467,30.676329,51.449275,4.589372,13.285024,2.23,1.994681,9.255663,4.739336,5.873715,4.579662,3.359031,18.355360,6.057269,7.066814,NaN,0.220264,20.392805,30.809471,376.829268,1545.0,686.0,2.234259,5.89,14.0,9.0,308.0,331.0,1150000.0,100.0,penthouse,0.0,5.0,NaN,NaN,1.0,2.0
1,On request,0.0,0.0,242.0,0.511176,0.286451,0.090908,0.0,0.0,0.0,0.058298,0.067048,0.103850,0.092914,0.209530,0.366674,0.082170,0.001811,0.011871,0.030169,0.052120,0.098951,3.038467,30.676329,51.449275,4.589372,13.285024,2.23,1.994681,9.255663,4.739336,5.873715,4.579662,3.359031,18.355360,6.057269,7.066814,NaN,0.220264,20.392805,30.809471,376.829268,1545.0,686.0,2.234259,5.89,14.0,9.0,308.0,331.0,1420000.0,156.0,terrace-house,222.0,5.0,NaN,NaN,1.0,2.0
2,Immediately,2.0,0.0,NaN,0.163362,0.095877,0.001911,0.0,0.0,0.0,0.334957,0.381257,0.297575,0.325887,0.393783,0.635194,0.154274,0.188229,0.000000,0.172646,0.163850,0.165830,0.909587,11.354420,32.197891,7.137064,49.310624,2.01,2.023799,21.358623,3.814582,3.633134,5.324421,3.782202,18.089552,7.899807,8.851305,NaN,0.735032,26.515854,22.662290,1704.700162,21036.0,10149.0,3.549010,6.05,37.0,3092.0,30364.0,33493.0,720000.0,93.0,penthouse,0.0,5.0,NaN,NaN,1.0,2.0
3,On request,0.0,0.0,257.0,0.511176,0.286451,0.090908,0.0,0.0,0.0,0.058298,0.067048,0.103850,0.092914,0.209530,0.366674,0.082170,0.001811,0.011871,0.030169,0.052120,0.098951,3.038467,30.676329,51.449275,4.589372,13.285024,2.23,1.994681,9.255663,4.739336,5.873715,4.579662,3.359031,18.355360,6.057269,7.066814,NaN,0.220264,20.392805,30.809471,376.829268,1545.0,686.0,2.234259,5.89,14.0,9.0,308.0,331.0,1430000.0,154.0,detached-house,370.0,5.0,NaN,NaN,1.0,2.0
4,On request,0.0,0.0,NaN,0.333865,0.279276,0.145835,0.0,0.0,0.0,0.133498,0.132933,0.235917,0.190986,0.136984,0.204549,0.109586,0.141473,0.091805,0.046950,0.038008,0.055509,1.460245,33.137090,49.705635,1.177460,15.979815,2.28,0.691563,15.901990,1.160862,5.217740,5.728026,5.006679,19.158429,6.502805,7.477959,NaN,0.892332,20.459524,27.590168,511.008403,6081.0,2638.0,1.708126,6.30,65.0,349.0,941.0,1355.0,995000.0,142.0,flat,0.0,5.0,NaN,NaN,1.0,2.0


In [17]:
#drop column Availability
df = df.drop(columns=["Availability"])

In [28]:
# Define categorical and numerical columns
cat_col = ["type_unified", "region_group"]
num_col = [col for col in df.columns if col not in cat_col + ["price_cleaned"]]

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessors in a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_col),
        ('cat', categorical_transformer, cat_col)
    ]
)

# Random Forest baseline model

In [19]:
rf_baseline = RandomForestRegressor(
    n_estimators=100,
    criterion='squared_error',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features=1.0,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=42,
    verbose=0,
    warm_start=False,
    ccp_alpha=0.0,
    max_samples=None,
    monotonic_cst=None
)

In [20]:
rf_estimater_10 = RandomForestRegressor(
    n_estimators=10,
    criterion='squared_error',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features=1.0,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=42,
    verbose=0,
    warm_start=False,
    ccp_alpha=0.0,
    max_samples=None,
    monotonic_cst=None
)

In [21]:
# Create and combine preprocessing and modeling in a pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', rf_baseline)
])

# Separate target and features
X = df.drop(columns=["price_cleaned"])
y = df["price_cleaned"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Floor',
                                                   'detail_responsive#surface_usable',
                                                   'Floor_space_merged',
                                                   'ForestDensityL',
                                                   'ForestDensityM',
                                                   'ForestDensityS',
                                                   'NoisePollutionRailwayL',
                                                   'NoisePollutionRailwayM',
                                                   'NoisePollutionRailwayS',
                                                   'NoisePollutionRo...
                                                   'gde_area_nonproductive_percentage',
                                                   'gde_area_settlement_percentage',
                                                   'gde_average_house_hold',
                                                   'gde_empty_apartments',
                                                   'gde_foreigners_percentage',
                                                   'gde_new_homes_per_1000', ...]),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['type_unified',
                                                   'region_group'])])),
                ('model', RandomForestRegressor(random_state=42))])

In [22]:
#caluclate mape on train data
y_pred = pipeline.predict(X_train)
r2, R2_adjusted, mape = calculate_metrics(X_train, y_train, y_pred)
print(f"R2 {r2},Adjusted R^2:{R2_adjusted}, MAPE:{mape}%")

R2 0.9689,Adjusted R^2:0.9688, MAPE:11.7605%


In [23]:
#caluclate mape on test data
y_pred = pipeline.predict(X_test)
r2, R2_adjusted, mape=calculate_metrics(X_train, y_test, y_pred)
print(f"R2 {r2},Adjusted R^2:{R2_adjusted}, MAPE:{mape}%")

R2 0.7036,Adjusted R^2:0.6995, MAPE:30.3831%


In [ ]:
estimators = [10, 20, 50, 70, 100, 200]

# Store average MAPE scores and their standard deviations for train and test
train_mape_scores_mean = []
train_mape_scores_std = []
test_mape_scores_mean = []
test_mape_scores_std = []

# Custom scorer for cross_val_score
mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

# Define cross-validation strategy
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Iterate over the different numbers of estimators
for estimator in tqdm(estimators):
    rf_estimator = RandomForestRegressor(
        n_estimators=estimator,
        criterion='squared_error',
        random_state=42,
        n_jobs=-1
    )
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', rf_estimator)
    ])

    # Perform cross-validation on the training dataset
    train_mape = -cross_val_score(
        pipeline,
        X_train,
        y_train,
        cv=cv,
        scoring=mape_scorer
    )

    # Fit the pipeline and predict on the test set for each split
    test_mape = []
    for train_idx, test_idx in cv.split(X_train):
        pipeline.fit(X_train.iloc[train_idx], y_train.iloc[train_idx])
        y_test_pred = pipeline.predict(X_train.iloc[test_idx])
        test_mape.append(mean_absolute_percentage_error(y_train.iloc[test_idx], y_test_pred))


    # Calculate mean and standard deviation for training and testing MAPE
    train_mape_scores_mean.append(np.mean(train_mape) * 100)
    train_mape_scores_std.append(np.std(train_mape) * 100)
    test_mape_scores_mean.append(np.mean(test_mape) * 100)
    test_mape_scores_std.append(np.std(test_mape) * 100)

    print(f"Estimator: {estimator}, Train MAPE: {np.mean(train_mape) * 100:.2f}% (±{np.std(train_mape) * 100:.2f}), "
          f"Test MAPE: {np.mean(test_mape) * 100:.2f}% (±{np.std(test_mape) * 100:.2f})")


  0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
# Plot the MAPE scores
plt.figure(figsize=(10, 6))
plt.errorbar(estimators, train_mape_scores_mean, yerr=train_mape_scores_std, label='Train MAPE', marker='o', capsize=5)
plt.errorbar(estimators, test_mape_scores_mean, yerr=test_mape_scores_std, label='Test MAPE', marker='o', capsize=5)
plt.xlabel('Number of Estimators')
plt.ylabel('MAPE (%)')
plt.title('Train vs Test MAPE Scores for Random Forest')
plt.legend()
plt.grid(True)
plt.show()

